In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
"""
 The Zillow csv provides ZHVI for neighborhoods in the US.
 Zillow Home Value Index (ZHVI): A measure of the typical home value and market changes across a given region and housing type. 
 It reflects the typical value for homes in the 35th to 65th percentile range.
 More info about ZHVI: https://www.zillow.com/research/methodology-neural-zhvi-32128/
"""

df = pd.read_csv('Neighborhood_zillow.csv')
print("shape:", df.shape)

In [ ]:
# print columns
print("Columns:")
print(df.columns.to_list())

chicago_df = df[df['City'] == "Chicago"]

# Different neighborhoods
print()
print("Chicago neighborhoods:")
print(chicago_df['RegionName'].unique()[:15])

# isolate Little Italy
print()
little_italy = chicago_df[chicago_df['RegionName'] == 'University Village - Little Italy']
print("Little Italy 2000-01", little_italy['2000-01-31'])
print("Little Italy 2024-09", little_italy['2024-09-30'])


In [ ]:
# Filter regions into different dataframes based on classification

# North Side
north_side_regions = [
    'Rogers Park', 'Edgewater', 'Uptown', 'Lake View', 'Lincoln Park', 
    'North Center', 'Lincoln Square', 'West Ridge', 'Irving Park', 
    'Albany Park', 'Avondale'
]
north_side_df = chicago_df[chicago_df['RegionName'].isin(north_side_regions)]

# South Side
south_side_regions = [
    'Armour Square', 'Bridgeport', 'Brighton Park', 'New City (Back of the Yards)', 
    'Englewood', 'Greater Grand Crossing', 'Hyde Park', 'Kenwood', 'Oakland', 
    'South Shore', 'Washington Park', 'Woodlawn', 'Chatham', 'South Chicago', 
    'Auburn Gresham', 'Calumet Heights', 'Roseland', 'Pullman', 'West Pullman', 
    'Riverdale'
]
south_side_df = chicago_df[chicago_df['RegionName'].isin(south_side_regions)]

# East Side
east_side_regions = ['Hegewisch', 'East Side', 'South Shore', 'Hyde Park', 'Kenwood']
east_side_df = chicago_df[chicago_df['RegionName'].isin(east_side_regions)]

# West Side
west_side_regions = [
    'Austin', 'East Garfield Park', 'West Garfield Park', 'North Lawndale', 
    'South Lawndale (Little Village)', 'Humboldt Park', 'Near West Side', 'West Town'
]
west_side_df = chicago_df[chicago_df['RegionName'].isin(west_side_regions)]

# Northwest Side
northwest_side_regions = [
    'Jefferson Park', 'Portage Park', 'Norwood Park', 'Dunning', 'Belmont Cragin', 
    'Montclare', 'Irving Park', 'Hermosa'
]
northwest_side_df = chicago_df[chicago_df['RegionName'].isin(northwest_side_regions)]

# Southwest Side
southwest_side_regions = [
    'Garfield Ridge', 'Archer Heights', 'Brighton Park', 'Gage Park', 'West Elsdon', 
    'West Lawn', 'Chicago Lawn (Marquette Park)', 'Ashburn', 'Clearing'
]
southwest_side_df = chicago_df[chicago_df['RegionName'].isin(southwest_side_regions)]

print("North Side neighborhoods: \n", north_side_df.head())
print("South Side neighborhoods: \n", south_side_df.head())
print("East Side neighborhoods: \n", east_side_df.head())
print("West Side neighborhoods: \n", west_side_df.head())
print("Northwest Side neighborhoods: \n", northwest_side_df.head())
print("Southwest Side neighborhoods: \n", southwest_side_df.head())

In [ ]:
# Unpivot date columns so data is more rectangular
def reshape_dates(df):
    # Identify date columns 
    date_columns = [col for col in df.columns if col.startswith('20')]
    
    # Melt dataframe to convert date columns into rows
    df_melted = df.melt(id_vars=['RegionID', 'SizeRank', 'RegionName'],
                        value_vars=date_columns,
                        var_name='Date', value_name='ZHVI')
    
    # Convert Date to datetime fromat
    df_melted['Date'] = pd.to_datetime(df_melted['Date'])

    return df_melted

reshaped_df = reshape_dates(chicago_df)
reshaped_df.head()

In [ ]:
# Filter for Lake View
lake_view_df = reshaped_df[reshaped_df['RegionName'] == 'Lake View']

# Plot ZHVI over time for Lake View
plt.figure(figsize=(12, 6))
sns.lineplot(x='Date', y='ZHVI', data=lake_view_df)
plt.title("ZHVI Value Over Time for Lake View")
plt.xlabel("Date")
plt.ylabel("ZHVI")
plt.show()

In [ ]:
cens = pd.read_csv('Data/Census-2020.csv')
cens['Label (Grouping)'].to_list

In [ ]:
def clean_census(year):
    filename = 'Data/Census-' + str(year) + '.csv'
    raw_df = pd.read_csv(filename)

    # Filter only Total Estimate columns
    columns = raw_df.columns.str
    df = raw_df.loc[:, columns.contains('Total!!Estimate')]
    df.loc[:,'Label'] = raw_df['Label (Grouping)']

    # Transpose columns and rows
    df = df.set_index('Label').transpose().reset_index()

    # Rename columns that have Year in the name
    df = df.rename(columns={'index': 'ZIP Code', 
                            'INCOME IN THE PAST 12 MONTHS (IN ' + str(year) + ' INFLATION-ADJUSTED DOLLARS)':
                            'INCOME IN THE PAST 12 MONTHS (IN INFLATION-ADJUSTED DOLLARS)', 
                            'EARNINGS IN THE PAST 12 MONTHS (IN ' + str(year) + ' INFLATION-ADJUSTED DOLLARS) FOR FULL-TIME, YEAR-ROUND WORKERS': 
                            'EARNINGS IN THE PAST 12 MONTHS (IN INFLATION-ADJUSTED DOLLARS) FOR FULL-TIME, YEAR-ROUND WORKERS'})
    
    
    # Rename columns so each is unique
    new_columns = []
    prefix = ''

    for col in df.columns:
        col = col.replace('\xa0', '')
        if col.isupper():
            # If the column name is all uppercase, set it as the prefix
            prefix = col
            new_columns.append(col)
        else:
            # Add the prefix to the column name
            new_columns.append(f"{prefix} - {col}")
    
    df.columns = new_columns
    df.columns = df.columns.str.strip()

    # Remove columns with all NaN values
    df = df.dropna(axis=1, how='all')

    # This column was causing trouble so remove it
    for col in df.columns:
        if 'with related children' in col.lower():
            df.drop(columns=[col])

    df = df.rename(columns={'- ZIP Code': 'ZIP Code', '- Total population': 'Total population'})
    df['ZIP Code'] = df['ZIP Code'].str[6:11]

    for col in df.columns:
        try:
            # Check if the column is of type 'object'
            if df[col].dtype == 'object':

                # Remove commas
                df[col] = df[col].replace({',': ''}, regex=True)
                
                # Check if the column contains percentages
                if df[col].str.contains('%').any():
                    # Convert to float after removing % sign and divide by 100
                    df[col] = df[col].str.rstrip('%').astype('float') / 100
                else:
                    df[col] = pd.to_numeric(df[col])

        except Exception as e:
            nonee = ''

    # Clean up column names and add year column
    df['Year'] = year

    return df


census_df = clean_census(2011)

# Loop through each year and add it to the census_df
for year in range(2012,2023):
    df = clean_census(year)

    # Remove columns not in the intersection so they can be concated
    common_columns = census_df.columns.intersection(df.columns)
    census_df = census_df[common_columns]
    df = df[common_columns]

    census_df = pd.concat([census_df, df], ignore_index=True)

census_df['ZIP Code'] = census_df['ZIP Code'].astype(str)

# Set display options to show all rows and columns
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
census_df

In [ ]:
# print(census_df.dtypes)
# Group by Zip and get average of population
grouped_df = census_df.groupby('ZIP Code')['Total population'].mean().reset_index()

# Get the top 10 ZIP codes by average population
top_10 = grouped_df.sort_values('Total population', ascending=False).head(10)

plt.figure(figsize=(12, 6))
sns.barplot(x='ZIP Code', y='Total population', data=top_10)
plt.title('Top 10 ZIP Codes by Average Total Population')
plt.show()

In [ ]:
# Clean up populations csv's
def clean_population(year):
    filename = 'Chicago_ZIP_Populations_' + str(year) + '.csv'
    df = pd.read_csv(filename)

    # Transpose columns and rows
    df = df.set_index('Label (Grouping)').transpose().reset_index()

    # Rename columns
    df.columns = ['Zip Code', 'Total']

    # Remove prefix from Zip codes
    df['Zip Code'] = df['Zip Code'].str[6:]
    df['Year'] = year
    
    return df

pop_2010 = clean_population(2010)
pop_2020 = clean_population(2020)

# Combine both years into one dataframe
population_df = pd.concat([pop_2010, pop_2020], ignore_index=True)

population_df

In [ ]:
# north side 

reshaped_north_side_df = reshape_dates(north_side_df)

# Plot ZHVI over time for Lake View
plt.figure(figsize=(12, 6))
sns.lineplot(x='Date', y='ZHVI', data=reshaped_north_side_df)
plt.title("ZHVI Value Over Time for North Side")
plt.xlabel("Date")
plt.ylabel("ZHVI")
plt.show()